In [1]:
import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

Train model on wind and energy data at once

In [ ]:
import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

columns_to_add = list()

for col in merged_hornsea.columns:
    if col not in merged_pes.columns:
        columns_to_add.append(col)

merged_all = merged_pes.merge(merged_hornsea[columns_to_add], left_index = True, right_index = True)
merged_all["energy"] = merged_all["Solar_MWh_credit"] + merged_all["Wind_MWh_credit"]
merged_all.drop(["Solar_MWh_credit", "Wind_MWh_credit"], axis = 1, inplace = True)

from Preprocessing import *
feature_engineerer_all = FeatureEngineerer(labels_to_remove=  ["energy"], columns_to_ohe = ['unavailabilityType', 'affectedUnit'], label = ["energy"])
feature_engineerer_all.perform_feature_engineering(merged_all, deployment = False, labels_to_remove=  ["energy"])

import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "lgbm_mode"

lgbm_model = model_utils.LGBMRegressorModel(feature_engineerer_all, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model.train_and_predict()  # This will skip training for already loaded models

def pinball_score(self):
        """pinball score implemetation"""

        score = []
        try:
            df = pd.DataFrame(self.q_predictions)
        except:
            df = pd.DataFrame()
            for k in self.q_predictions.keys():
                df[k] = pd.DataFrame(self.q_predictions[k])
        for qu in self.quantiles:
            score.append(self.pinball(y=df["true"], q=df[str(qu)], alpha=qu).mean())
        return sum(score) / len(score)

print(pinball_score(lgbm_model))
# import plotly.express as px
# df.index = feature_engineerer_all.y_test.index
# px.line(df, y = ["true", "0.5"])

In [ ]:
from Preprocessing import * 

prep = Preprocessing()

merged_hornsea = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_hornsea":"nc_files", "ncep_gfs_hornsea":"nc_files"},
                                              deployment = False, merge_with_outage_data = True, json_file_path = "nc_files/REMIT", energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

merged_pes = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_pes":"nc_files", "ncep_gfs_pes":"nc_files"},
          deployment = False, merge_with_outage_data = False, energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, ) 

merged_demand = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_demand":"nc_files", "ncep_gfs_demand":"nc_files"},
          deployment = False, merge_with_outage_data = True, energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

In [ ]:
merged_demand = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_demand":"nc_files", "ncep_gfs_demand":"nc_files"},
          deployment = False, merge_with_outage_data = True, energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

In [15]:
merged_hornsea.to_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes.to_parquet("preprocessed_pes_with_energy.parquet")

In [ ]:
from Preprocessing import * 

prep = Preprocessing()

merged_hornsea = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_hornsea":"nc_files", "ncep_gfs_hornsea":"nc_files"},
                                              deployment = False, merge_with_outage_data = True, json_file_path = "nc_files/REMIT", energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

merged_pes = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_pes":"nc_files", "ncep_gfs_pes":"nc_files"},
          deployment = False, merge_with_outage_data = False, energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, ) 

merged_demand = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_demand":"nc_files", "ncep_gfs_demand":"nc_files"},
          deployment = False, merge_with_outage_data = True, energy_data_dict = {"Energy_data":"csv_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

In [21]:
import importlib
import Preprocessing
importlib.reload(Preprocessing)
import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

feature_engineerer_wind = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe = ['unavailabilityType', 'affectedUnit'])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea.drop(["rel_hum", "rel_hum_diff", "forecast_horizon"], axis = 1), deployment = False)

feature_engineerer_solar = Preprocessing.FeatureEngineerer(label = "Solar_MWh_credit")
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_solar_baseline = Preprocessing.FeatureEngineerer(label = 'Solar_MWh_credit')
feature_engineerer_solar_baseline.perform_feature_engineering(merged_pes_simple, deployment = False)

merged_hornsea_simple = merged_hornsea[['wind_speed_100', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_wind_baseline = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit')
feature_engineerer_wind_baseline.perform_feature_engineering(merged_hornsea_simple, deployment = False)

In [1]:
import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

In [ ]:
print(f"Quantile Regressor Pinball Score: {lgbm_model.pinball_score()}")

In [31]:
df = pinball_score(lg2)

In [8]:
from model_utils import *
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from random import randint
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn.metrics import make_scorer, mean_pinball_loss
quantiles = np.arange(0.1, 1.0, 0.1).round(2)
model_save_dir_qr = "lgbm_model_wind"



# Define pinball loss function
def pinball_loss(y_true, y_pred, quantile=0.9):
    delta = y_true - y_pred
    loss = np.where(delta > 0, quantile * delta, (1 - quantile) * - delta)
    return np.mean(loss)

# Create a custom scorer for quantile regression (e.g., 90th percentile)
quantile = 0.9
scorer = make_scorer(pinball_loss, greater_is_better=False, quantile=quantile)

class LG(model_utils.LGBMRegressorModel):
    def __init__(self, feature_engineerer, quantiles, model_save_dir, load_pretrained):
        super().__init__(feature_engineerer, quantiles, model_save_dir, load_pretrained)
        self.best_params = dict()

    def train_and_predict(self):
        """Train the LGBMRegressor models or use the pretrained ones."""
        for quantile in self.quantiles:
            if not self.load_pretrained or quantile not in self.models:

                # Create a custom scorer for quantile regression (e.g., 90th percentile)
                quantile_now = quantile
                scorer = make_scorer(pinball_loss, quantile=quantile_now)
                

                params = {
                    "alpha":[quantile],
                    'boosting': ["gbdt", "dart"],
                    "force_col_wise": [True],
                    "num_iterations": [100, 200, 500],
                    "num_leaves": np.linspace(20, 100, 10, dtype = int).tolist(),
                    "max_depth": np.linspace(5, 15, 6, dtype = int).tolist(),
                    'learning_rate': np.logspace(-3, -1, 10).round(4).tolist(),
                    "n_estimators": [500, 1000, 1500],
                    #"subsample_for_bin": [],
                    'objective': ['quantile'],
                    #"min_split_gain": [],
                    #"min_sum_hessian_in_leaf:[] #for overfitting"
                    #"min_child_weight": [45],
                    # "min_child_samples":[20],
                    "min_data_in_leaf":np.linspace(50, 500, 20, dtype = int).tolist(),
                    #"tree_trainer":["serial", "feature", "data", "voting"],
                    # "feature_fraction_bynode": [1.0, 0.8, 0.5],
                    # "feature_fraction":[0.5, 0.8, 1.0],
                    # "bagging_fraction":[0.5, 0.8, 1.0],
                    #"subsample": [],
                    #"subsample_freq": [],
                    # "colsample_bytree": [0.3, 0.5, 1],
                    # "reg_alpha": np.logspace(-3, -1, 10).tolist(),
                    # "reg_lambda": np.logspace(-2, -1, 10).tolist(),
                    "random_state": [42],
                    "verbose":[-1],
                    #"device_type":["cuda"],
                    "extra_trees":[False],
                    # "path_smooth":[0.0, 0.2]
                    #"importance_type":[]
                }
                # Train a new model for this quantile
                # qr_lgbm1 = lgb.LGBMRegressor(objective='quantile', alpha=quantile, n_estimators=1000, force_col_wise=True, params = params)
                qr_lgbm1 = lgb.LGBMRegressor()
                cv = TimeSeriesSplit(2)
                qr_lgbm = RandomizedSearchCV(qr_lgbm1, param_distributions = params, cv = cv, n_iter = 5, 
                                             #scoring = scorer
                                             )
                qr_lgbm.fit(
                    self.feature_engineerer.X_train, 
                    self.feature_engineerer.y_train,
                    eval_set=[(self.feature_engineerer.X_train, self.feature_engineerer.y_train), 
                              (self.feature_engineerer.X_val, self.feature_engineerer.y_val)],
                    eval_names=['train', 'valid'],
                    eval_metric='quantile',
                    callbacks=[early_stopping(stopping_rounds=20), log_evaluation(50)]
                )

                # Save the model
                model_filename = os.path.join(self.model_save_dir, f"lgbm_model_quantile_{quantile}.pkl")
                self.best_params[quantile] = qr_lgbm.best_estimator_
                joblib.dump(qr_lgbm.best_estimator_, model_filename)
                print(f"Saved Quantile Regressor model for quantile {quantile} to {model_filename}")

                # Store the model for prediction
                self.models[quantile] = qr_lgbm.best_estimator_
            else:
                print(f"Using the loaded pretrained Quantile Regressor model for quantile {quantile}")

            # Predict and store the results
            self.q_predictions[str(quantile)] = self.models[quantile].predict(self.feature_engineerer.X_test)

        #sort results
        self.q_predictions = self.sort_quantiles(self.q_predictions, self.quantiles)
        self.q_predictions = self.replace_neg_values(self.q_predictions, self.quantiles)

        # **Set models_loaded to True after training so predict can be called immediately**
        self.models_loaded = True


lg2 = LG(feature_engineerer_all, quantiles, model_save_dir = model_save_dir_qr, load_pretrained = False)
lg2.train_and_predict()

c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 20 rounds
[50]	train's quantile: 34.6488	valid's quantile: 36.2569
[100]	train's quantile: 31.327	valid's quantile: 33.1
[150]	train's quantile: 28.6881	valid's quantile: 30.659
[200]	train's quantile: 26.5795	valid's quantile: 28.709
[250]	train's quantile: 24.754	valid's quantile: 27.1034
[300]	train's quantile: 23.2089	valid's quantile: 25.7572
[350]	train's quantile: 21.9963	valid's quantile: 24.6581
[400]	train's quantile: 21.0421	valid's quantile: 23.8594
[450]	train's quantile: 20.2151	valid's quantile: 23.0965
[500]	train's quantile: 19.5426	valid's quantile: 22.4842
Did not meet early stopping. Best iteration is:
[500]	train's quantile: 19.5426	valid's quantile: 22.4842


c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 20 rounds
[50]	train's quantile: 34.8657	valid's quantile: 36.4649
[100]	train's quantile: 31.9485	valid's quantile: 33.711
[150]	train's quantile: 28.9332	valid's quantile: 30.8622
[200]	train's quantile: 26.4179	valid's quantile: 28.5119
[250]	train's quantile: 24.262	valid's quantile: 26.5269
[300]	train's quantile: 22.4277	valid's quantile: 24.8366
[350]	train's quantile: 20.9191	valid's quantile: 23.4503
[400]	train's quantile: 19.7567	valid's quantile: 22.3498
[450]	train's quantile: 18.7553	valid's quantile: 21.4203
[500]	train's quantile: 17.9417	valid's quantile: 20.6797
Did not meet early stopping. Best iteration is:
[500]	train's quantile: 17.9417	valid's quantile: 20.6797
[50]	train's quantile: 35.7188	valid's quantile: 37.2002
[100]	train's quantile: 34.7697	valid's quantile: 36.2332


c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\callback.py:329: UserWarning: Early stopping is not available in dart mode
  _log_warning("Early stopping is not available in dart mode")


...

Training until validation scores don't improve for 20 rounds
[50]	train's quantile: 30.0335	valid's quantile: 33.2473
[100]	train's quantile: 23.7634	valid's quantile: 26.9504
[150]	train's quantile: 18.2179	valid's quantile: 21.7956
[200]	train's quantile: 15.5065	valid's quantile: 19.4312
[250]	train's quantile: 14.3038	valid's quantile: 18.465
[300]	train's quantile: 13.861	valid's quantile: 18.2076
[350]	train's quantile: 13.573	valid's quantile: 18.1468
[400]	train's quantile: 13.4263	valid's quantile: 18.2822
[450]	train's quantile: 13.3312	valid's quantile: 18.3495
[500]	train's quantile: 13.3065	valid's quantile: 18.4699


c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 20 rounds
[50]	train's quantile: 28.462	valid's quantile: 30.9768
[100]	train's quantile: 21.0607	valid's quantile: 24.2684
[150]	train's quantile: 16.3679	valid's quantile: 20.8523
[200]	train's quantile: 14.1039	valid's quantile: 19.1086
[250]	train's quantile: 13.1434	valid's quantile: 18.439
[300]	train's quantile: 13.5283	valid's quantile: 18.379
[350]	train's quantile: 14.4831	valid's quantile: 18.7393
[400]	train's quantile: 14.4518	valid's quantile: 18.8421
[450]	train's quantile: 14.3632	valid's quantile: 18.8399
[500]	train's quantile: 14.3301	valid's quantile: 18.934
Training until validation scores don't improve for 20 rounds
[50]	train's quantile: 28.8859	valid's quantile: 30.9439


c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\danie\anaconda3\envs\awp2\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[100]	train's quantile: 23.2514	valid's quantile: 25.3452
[150]	train's quantile: 16.5735	valid's quantile: 19.3762
[200]	train's quantile: 13.0872	valid's quantile: 16.5292
[250]	train's quantile: 11.4826	valid's quantile: 15.3766
[300]	train's quantile: 10.7351	valid's quantile: 15.0626
[350]	train's quantile: 10.3219	valid's quantile: 14.9764
[400]	train's quantile: 10.0942	valid's quantile: 14.9916
[450]	train's quantile: 9.88054	valid's quantile: 14.9878
[500]	train's quantile: 9.72276	valid's quantile: 15.0306
Saved Quantile Regressor model for quantile 0.9 to lgbm_model_wind\lgbm_model_quantile_0.9.pkl


In [16]:
merged_all["energy"] = merged_all["Solar_MWh_credit"] + merged_all["Wind_MWh_credit"]
merged_all.drop(["Solar_MWh_credit", "Wind_MWh_credit"], axis = 1, inplace = True)

In [2]:
feature_engineerer_wind.train_val_test_split_by_year(merged_hornsea)

In [ ]:
lgbm_model_wind.pinball_score()

In [ ]:
importlib.reload(model_utils)
import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "lgbm_model_wind"

lgbm_model_wind = model_utils.LGBMRegressorModel(feature_engineerer_wind, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_wind.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_wind.pinball_score()}")

In [ ]:
lgbm_model_wind.pinball_score()

In [ ]:
importlib.reload(model_utils)
import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "lgbm_model_wind"

lgbm_model_wind = model_utils.LGBMRegressorModel(feature_engineerer_wind, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_wind.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_wind.pinball_score()}")

In [ ]:
lgbm_model_solar.pinball_score()

In [ ]:
importlib.reload(model_utils)
import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "lgbm_model_solar"

lgbm_model_solar = model_utils.LGBMRegressorModel(feature_engineerer_solar, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_solar.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_solar.pinball_score()}")

In [ ]:
importlib.reload(model_utils)
import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "lgbm_model_wind"

lgbm_model_wind = model_utils.LGBMRegressorModel(feature_engineerer_wind, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_wind.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_wind.pinball_score()}")

In [ ]:
lgbm_model_wind.models

In [ ]:
lgbm_model_wind.q_predictions

In [ ]:
lg2.pinball_score()

In [ ]:
lg2.best_params

In [1]:
import importlib
import Preprocessing
importlib.reload(Preprocessing)
import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

feature_engineerer_wind = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe = ['unavailabilityType', 'affectedUnit'])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea.drop(["rel_hum", "rel_hum_diff"], axis = 1), deployment = False)

feature_engineerer_solar = Preprocessing.FeatureEngineerer(label = "Solar_MWh_credit")
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_solar_baseline = Preprocessing.FeatureEngineerer(label = 'Solar_MWh_credit')
feature_engineerer_solar_baseline.perform_feature_engineering(merged_pes_simple, deployment = False)

merged_hornsea_simple = merged_hornsea[['wind_speed_100', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_wind_baseline = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit')
feature_engineerer_wind_baseline.perform_feature_engineering(merged_hornsea_simple, deployment = False)

In [ ]:
from model_utils import *
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from random import randint
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn.metrics import make_scorer, mean_pinball_loss
quantiles = np.arange(0.1, 1.0, 0.1).round(2)
model_save_dir_qr = "lgbm_model_wind"



# Define pinball loss function
def pinball_loss(y_true, y_pred, quantile=0.9):
    delta = y_true - y_pred
    loss = np.where(delta > 0, quantile * delta, (1 - quantile) * - delta)
    return np.mean(loss)

# Create a custom scorer for quantile regression (e.g., 90th percentile)
quantile = 0.9
scorer = make_scorer(pinball_loss, greater_is_better=False, quantile=quantile)

class LG(model_utils.LGBMRegressorModel):
    def __init__(self, feature_engineerer, quantiles, model_save_dir, load_pretrained):
        super().__init__(feature_engineerer, quantiles, model_save_dir, load_pretrained)
        self.best_params = dict()

    def train_and_predict(self):
        """Train the LGBMRegressor models or use the pretrained ones."""
        for quantile in self.quantiles:
            if not self.load_pretrained or quantile not in self.models:

                # Create a custom scorer for quantile regression (e.g., 90th percentile)
                quantile_now = quantile
                scorer = make_scorer(pinball_loss, quantile=quantile_now)
                

                params = {
                    "alpha":[quantile],
                    'boosting': ["gbdt", "dart"],
                    "force_col_wise": [True],
                    "num_iterations": [100, 200, 500],
                    "num_leaves": np.linspace(20, 100, 10, dtype = int).tolist(),
                    "max_depth": np.linspace(5, 15, 6, dtype = int).tolist(),
                    'learning_rate': np.logspace(-3, -1, 10).round(4).tolist(),
                    "n_estimators": [500, 1000, 1500],
                    #"subsample_for_bin": [],
                    'objective': ['quantile'],
                    #"min_split_gain": [],
                    #"min_sum_hessian_in_leaf:[] #for overfitting"
                    #"min_child_weight": [45],
                    # "min_child_samples":[20],
                    "min_data_in_leaf":np.linspace(50, 500, 20, dtype = int).tolist(),
                    #"tree_trainer":["serial", "feature", "data", "voting"],
                    # "feature_fraction_bynode": [1.0, 0.8, 0.5],
                    # "feature_fraction":[0.5, 0.8, 1.0],
                    # "bagging_fraction":[0.5, 0.8, 1.0],
                    #"subsample": [],
                    #"subsample_freq": [],
                    # "colsample_bytree": [0.3, 0.5, 1],
                    # "reg_alpha": np.logspace(-3, -1, 10).tolist(),
                    # "reg_lambda": np.logspace(-2, -1, 10).tolist(),
                    "random_state": [42],
                    "verbose":[-1],
                    #"device_type":["cuda"],
                    "extra_trees":[False],
                    # "path_smooth":[0.0, 0.2]
                    #"importance_type":[]
                }
                # Train a new model for this quantile
                # qr_lgbm1 = lgb.LGBMRegressor(objective='quantile', alpha=quantile, n_estimators=1000, force_col_wise=True, params = params)
                qr_lgbm1 = lgb.LGBMRegressor()
                cv = TimeSeriesSplit(2)
                qr_lgbm = RandomizedSearchCV(qr_lgbm1, param_distributions = params, cv = cv, n_iter = 10, 
                                             #scoring = scorer
                                             )
                qr_lgbm.fit(
                    self.feature_engineerer.X_train, 
                    self.feature_engineerer.y_train,
                    eval_set=[(self.feature_engineerer.X_train, self.feature_engineerer.y_train), 
                              (self.feature_engineerer.X_val, self.feature_engineerer.y_val)],
                    eval_names=['train', 'valid'],
                    eval_metric='quantile',
                    callbacks=[early_stopping(stopping_rounds=20), log_evaluation(50)]
                )

                # Save the model
                model_filename = os.path.join(self.model_save_dir, f"lgbm_model_quantile_{quantile}.pkl")
                self.best_params[quantile] = qr_lgbm.best_estimator_
                joblib.dump(qr_lgbm.best_estimator_, model_filename)
                print(f"Saved Quantile Regressor model for quantile {quantile} to {model_filename}")

                # Store the model for prediction
                self.models[quantile] = qr_lgbm.best_estimator_
            else:
                print(f"Using the loaded pretrained Quantile Regressor model for quantile {quantile}")

            # Predict and store the results
            self.q_predictions[str(quantile)] = self.models[quantile].predict(self.feature_engineerer.X_test)

        #sort results
        self.q_predictions = self.sort_quantiles(self.q_predictions, self.quantiles)
        self.q_predictions = self.replace_neg_values(self.q_predictions, self.quantiles)

        # **Set models_loaded to True after training so predict can be called immediately**
        self.models_loaded = True


lg2 = LG(feature_engineerer_wind, quantiles, model_save_dir = model_save_dir_qr, load_pretrained = False)
lg2.train_and_predict()

In [ ]:
def train_and_predict(self):
        """Train the LGBMRegressor models or use the pretrained ones."""
        for quantile in self.quantiles:
            if not self.load_pretrained or quantile not in self.models:
                # Train a new model for this quantile
                qr_lgbm = lgb.LGBMRegressor(objective='quantile', alpha=quantile, n_estimators=1000, force_col_wise=True)
                qr_lgbm.fit(
                    self.feature_engineerer.X_train, 
                    self.feature_engineerer.y_train,
                    eval_set=[(self.feature_engineerer.X_train, self.feature_engineerer.y_train), 
                              (self.feature_engineerer.X_val, self.feature_engineerer.y_val)],
                    eval_names=['train', 'valid'],
                    eval_metric='quantile',
                    callbacks=[early_stopping(stopping_rounds=50), log_evaluation(50)]
                )

                # Save the model
                model_filename = os.path.join(self.model_save_dir, f"lgbm_model_quantile_{quantile}.pkl")
                joblib.dump(qr_lgbm, model_filename)
                print(f"Saved Quantile Regressor model for quantile {quantile} to {model_filename}")

                # Store the model for prediction
                self.models[quantile] = qr_lgbm
            else:
                print(f"Using the loaded pretrained Quantile Regressor model for quantile {quantile}")

            # Predict and store the results
            self.q_predictions[str(quantile)] = self.models[quantile].predict(self.feature_engineerer.X_test)

        # **Set models_loaded to True after training so predict can be called immediately**
        self.models_loaded = True

In [ ]:
from Preprocessing import * 

prep = Preprocessing()

merged_pes2 = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_pes":"api_files", "ncep_gfs_pes":"api_files"},
          deployment = True, merge_with_outage_data = False, energy_data_dict = {"Energy_Data":"api_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

merged_hornsea2 = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_hornsea":"api_files", "ncep_gfs_hornsea":"api_files"},
          deployment = True, merge_with_outage_data = True, energy_data_dict = {"Energy_Data":"api_files"},
                                              non_numerical_columns = ["unavailabilityType", "affectedUnit"],
                                              fft = False, )

In [ ]:
merged_hornsea.columns

In [16]:
a = ['Solar_MWh_credit', 'Wind_MWh_credit', 'affectedUnit',
       'availableCapacity', 'cos_day', 'cos_dayofweek', 'cos_hour',
       'cos_month', 'forecast_horizon', 'hoursSinceOutage',
       'hoursUntilOutageEnd', 'outage', 'rel_hum', 'rel_hum_diff', 'sin_day',
       'sin_dayofweek', 'sin_hour', 'sin_month', 'temp', 'temp_diff',
       'temp_max_3h', 'temp_max_6h', 'temp_mean_3h', 'temp_mean_6h',
       'temp_min_3h', 'temp_min_6h', 'temp_next_forecast', 'temp_range_3h',
       'temp_std_3h', 'temp_std_6h', 'unavailabilityType',
       'unavailableCapacity', 'wind_dir_100_cos', 'wind_dir_100_sin',
       'wind_dir_cos', 'wind_dir_sin', 'wind_speed', 'wind_speed_100',
       'wind_speed_100_diff', 'wind_speed_100_max_3h', 'wind_speed_100_max_6h',
       'wind_speed_100_mean_3h', 'wind_speed_100_mean_6h',
       'wind_speed_100_min_3h', 'wind_speed_100_min_6h',
       'wind_speed_100_next_forecast', 'wind_speed_100_range_3h',
       'wind_speed_100_std_3h', 'wind_speed_100_std_6h',
       'wind_speed_altitude_diff', 'wind_speed_diff', 'wind_speed_max_3h',
       'wind_speed_max_6h', 'wind_speed_mean_3h', 'wind_speed_mean_6h',
       'wind_speed_min_3h', 'wind_speed_min_6h', 'wind_speed_next_forecast',
       'wind_speed_range_3h', 'wind_speed_std_3h', 'wind_speed_std_6h']

for col in a:
    if col not in merged_hornsea.columns:
        print(col)

In [11]:
feature_engineerer_wind.perform_feature_engineering(merged_hornsea2, deployment = True)

In [12]:
feature_engineerer_solar.perform_feature_engineering(merged_pes2, deployment = True)

### __Wind Energy Forecast__

__baseline__

In [ ]:
import model_utils
import numpy as np
import importlib
importlib.reload(model_utils)

quantiles = np.arange(0.1, 1.0, 0.1).round(2)

# Specify model save directory
model_save_dir_qr = "qr_model_wind"

qr_model_wind = model_utils.QuantileRegressorModel(feature_engineerer_wind_baseline, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=True)
qr_model_wind.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {qr_model_wind.pinball_score()}")

In [ ]:
qr_model_wind.plot_quantils(feature_engineerer_wind_baseline.y_test.index, qr_model_wind.q_predictions, quantiles=np.arange(0.1, 1.0, 0.1), year=2023)

__xgboost__

In [ ]:
importlib.reload(model_utils)
import numpy as np

quantiles = np.arange(0.1, 1.0, 0.1)

model_save_dir_xgboost = "xgboost_model_wind"
hyperparams = {
            # Use the quantile objective function.
            "objective": "reg:quantileerror",
            "tree_method": "hist",
            "quantile_alpha": quantiles,
            "learning_rate": 0.01,
            "max_depth": 8
        }

xgboost_model_wind = model_utils.XGBoostModel(feature_engineerer_wind, quantiles=quantiles, model_save_dir=model_save_dir_xgboost, load_pretrained=False, hyperparams=hyperparams, num_boost_round=45, early_stopping_rounds=10)
xgboost_model_wind.train_and_predict()  # This will skip training if the model is already loaded
print(f"XGBoost Pinball Score: {xgboost_model_wind.pinball_score()}")

In [ ]:
xgboost_model_wind.plot_feature_importance()

In [ ]:
xgboost_model_wind.plot_quantils(daterange=feature_engineerer_wind.y_test.index, y=xgboost_model_wind.q_predictions, quantiles=np.arange(0.1, 1.0, 0.1), year=2023)

### __Solar Energy Forecast__

__baseline modell__

In [ ]:
quantiles = np.arange(0.1, 1.0, 0.1)

# Specify model save directory
model_save_dir_qr = "qr_model_solar"

qr_model_solar = model_utils.QuantileRegressorModel(feature_engineerer_solar_baseline, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=True)
qr_model_solar.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {qr_model_solar.pinball_score()}")

In [ ]:
qr_model_solar.plot_quantils(daterange=feature_engineerer_solar_baseline.y_test.index, y=qr_model_solar.q_predictions, quantiles=np.arange(0.1, 1.0, 0.1), year=2023)

__xgboost__

In [ ]:
quantiles = np.arange(0.1, 1.0, 0.1)

model_save_dir_xgboost = "xgboost_model_solar"

xgboost_model_solar = model_utils.XGBoostModel(feature_engineerer_solar, quantiles=quantiles, model_save_dir=model_save_dir_xgboost, load_pretrained=False)
xgboost_model_solar.train_and_predict()  # This will skip training if the model is already loaded
print(f"XGBoost Pinball Score: {xgboost_model_solar.pinball_score()}")

In [ ]:
xgboost_model_solar.plot_feature_importance()

In [ ]:
xgboost_model_solar.plot_quantils(daterange=feature_engineerer_solar.y_test.index, y=xgboost_model_solar.q_predictions, quantiles=np.arange(0.1, 1.0, 0.1), year=2023, month = 7, day = 10)

### __Lightgbm implementation__

In [ ]:
importlib.reload(model_utils)
import numpy as np
quantiles = np.arange(0.1, 1.0, 0.1)

# Specify model save directory
model_save_dir_qr = "lgbm_model_wind"

lgbm_model_wind = model_utils.LGBMRegressorModel(feature_engineerer_wind, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_wind.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_wind.pinball_score()}")

In [ ]:
lgbm_model_wind.plot_quantils(feature_engineerer_wind.y_test.index, lgbm_model_wind.q_predictions, quantiles, year = 2023, month=7)

In [ ]:
lgbm_model_wind.plot_quantils(feature_engineerer_wind.y_test.index, lgbm_model_wind.q_predictions, quantiles, year = 2023, month=7)

In [ ]:
lgbm_model_wind.q_predictions

In [ ]:
quantiles = np.arange(0.1, 1.0, 0.1)

# Specify model save directory
model_save_dir_qr = "lgbm_model_solar"

lgbm_model_solar = model_utils.LGBMRegressorModel(feature_engineerer_solar, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
lgbm_model_solar.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {lgbm_model_solar.pinball_score()}")

In [ ]:
len(feature_engineerer_wind.features_after_fe)

In [17]:
for g in feature_engineerer_wind.features_after_fe:
    if g not in a:
        print(g)

In [ ]:
len(feature_engineerer_wind.features_after_fe)

In [15]:
a = ['availableCapacity',
 'cos_day',
 'cos_dayofweek',
 'cos_hour',
 'cos_month',
 'forecast_horizon',
 'hoursSinceOutage',
 'hoursUntilOutageEnd',
 'outage',
 'rel_hum',
 'rel_hum_diff',
 'sin_day',
 'sin_dayofweek',
 'sin_hour',
 'sin_month',
 'temp',
 'temp_diff',
 'temp_max_3h',
 'temp_max_6h',
 'temp_mean_3h',
 'temp_mean_6h',
 'temp_min_3h',
 'temp_min_6h',
 'temp_next_forecast',
 'temp_range_3h',
 'temp_std_3h',
 'temp_std_6h',
 'unavailableCapacity',
 'wind_dir_100_cos',
 'wind_dir_100_sin',
 'wind_dir_cos',
 'wind_dir_sin',
 'wind_speed',
 'wind_speed_100',
 'wind_speed_100_diff',
 'wind_speed_100_max_3h',
 'wind_speed_100_max_6h',
 'wind_speed_100_mean_3h',
 'wind_speed_100_mean_6h',
 'wind_speed_100_min_3h',
 'wind_speed_100_min_6h',
 'wind_speed_100_next_forecast',
 'wind_speed_100_range_3h',
 'wind_speed_100_std_3h',
 'wind_speed_100_std_6h',
 'wind_speed_altitude_diff',
 'wind_speed_diff',
 'wind_speed_max_3h',
 'wind_speed_max_6h',
 'wind_speed_mean_3h',
 'wind_speed_mean_6h',
 'wind_speed_min_3h',
 'wind_speed_min_6h',
 'wind_speed_next_forecast',
 'wind_speed_range_3h',
 'wind_speed_std_3h',
 'wind_speed_std_6h',
 'unavailabilityType_None',
 'unavailabilityType_Planned',
 'unavailabilityType_Unplanned',
 'affectedUnit_HOWAO-1',
 'affectedUnit_HOWAO-2',
 'affectedUnit_HOWAO-3',
 'affectedUnit_HOWBO-1',
 'affectedUnit_HOWBO-3',
 'affectedUnit_None']

In [ ]:
import pickle

# Assuming `model` is your trained model
with open('lgbm_model_wind.pkl', 'wb') as file:
    pickle.dump(lgbm_model_wind, file)
with open('lgbm_model_solar.pkl', 'wb') as file:
    pickle.dump(lgbm_model_solar, file)

In [ ]:
lgbm_model_solar.plot_quantils(feature_engineerer_solar.y_test.index, lgbm_model_solar.q_predictions, quantiles, year=2023, month=6, day=19)

In [3]:
class q_model2(neural_networks.q_model):
    def __init__(self):
        super().__init__(self)

q2 = q_model2

In [ ]:
import importlib
import Preprocessing
importlib.reload(Preprocessing)

import model_utils
import pandas as pd

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

feature_engineerer_wind = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe = ['unavailabilityType', 'affectedUnit'])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea, deployment = False)

feature_engineerer_solar = Preprocessing.FeatureEngineerer(label = "Solar_MWh_credit")
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_solar_baseline = Preprocessing.FeatureEngineerer(label = 'Solar_MWh_credit')
feature_engineerer_solar_baseline.perform_feature_engineering(merged_pes_simple, deployment = False)

merged_hornsea_simple = merged_hornsea[['wind_speed_100', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_wind_baseline = Preprocessing.FeatureEngineerer(label = 'Wind_MWh_credit')
feature_engineerer_wind_baseline.perform_feature_engineering(merged_hornsea_simple, deployment = False)

import neural_networks
import numpy as np
importlib.reload(neural_networks)
from functools import partial
from itertools import chain
import torch
import torch.nn as nn
quantiles = np.arange(0.1, 1.0, 0.1)

nn_wind = neural_networks.Trainer(feature_engineerer_wind, neural_networks.q_model,quantiles)
nn_wind.train_and_test()

In [ ]:
nn_wind.plot_quantils(feature_engineerer_wind.y_test.index, nn_wind.q_prediction_nn, quantiles, year = 2023, month = 9, day = 15)

In [ ]:
nn_wind.plot_quantils(feature_engineerer_wind.y_test.index, nn_wind.q_prediction_nn, quantiles, year = 2023, month = 6)

In [9]:
good_nn_1 = nn_wind

In [20]:
torch.save(nn_wind.model, "LSTM_Linear_Wind.pth")

In [ ]:
nn_wind.plot_quantils(feature_engineerer_wind.y_test.index, nn_wind.q_prediction_nn, quantiles, day=9)

In [ ]:
import neural_networks
importlib.reload(neural_networks)
from functools import partial
from itertools import chain
import torch
import torch.nn as nn
quantiles = np.arange(0.1, 1.0, 0.1)

nn_solar = neural_networks.Trainer(feature_engineerer_solar, neural_networks.q_model, quantiles, in_shape=30)
nn_solar.train_and_test()

In [ ]:
nn_solar.plot_quantils(feature_engineerer_solar.y_test.index, nn_solar.q_prediction_nn, quantiles, day=8)